In [ ]:
!pip uninstall torch transformers datasets wandb -y
!pip install torch transformers datasets
!pip install --upgrade torch transformers datasets

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: wandb 0.18.7
Uninstalling wandb-0.18.7:
  Successfully uninstalled wandb-0.18.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:

In [ ]:
import shutil

# Распаковка train_dataset
shutil.unpack_archive('train_dataset_ver2.zip', 'train_dataset')

# Распаковка test_dataset
shutil.unpack_archive('test_dataset_ver2.zip', 'test_dataset')

# Распаковка токенизатора
shutil.unpack_archive('tokenizer_ver2.zip', 'tokenizer')


In [ ]:
from datasets import load_from_disk

# Загрузка train и test датасетов
train_dataset = load_from_disk('train_dataset')
test_dataset = load_from_disk('test_dataset')


In [ ]:
from transformers import AutoTokenizer

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained('tokenizer')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# Загрузка модели
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3small_based_on_gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [ ]:
# Подготовка DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # не использует MLM
)

In [ ]:
# Настройки обучения
training_args = TrainingArguments(
    fp16=True,  # Включение смешанной точности
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5, #увеличина скорость обучения, было 5e-5
    per_device_train_batch_size=6, #выше не дает, ругается на ограничения
    num_train_epochs=2, #снижено до 2, было 3
    save_steps=10_000,
    save_total_limit=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Trainer
trainer = Trainer(
    model=model.to("cuda"),  # Убедитесь, что модель на GPU
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-10-c57a14cff95f>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Обучение модели
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.713900,3.596468
2,3.524500,3.567031


TrainOutput(global_step=4500, training_loss=3.681409016927083, metrics={'train_runtime': 1609.1531, 'train_samples_per_second': 16.779, 'train_steps_per_second': 2.797, 'total_flos': 7054884864000000.0, 'train_loss': 3.681409016927083, 'epoch': 2.0})

In [ ]:
# Создание директории для сохранения
output_dir = "./saved_model_ver2"

# Сохранение модели
trainer.save_model(output_dir)

# Сохранение токенизатора
tokenizer.save_pretrained(output_dir)

('./saved_model_ver2/tokenizer_config.json',
 './saved_model_ver2/special_tokens_map.json',
 './saved_model_ver2/vocab.json',
 './saved_model_ver2/merges.txt',
 './saved_model_ver2/added_tokens.json',
 './saved_model_ver2/tokenizer.json')

In [ ]:
import shutil

# Путь к папке, где сохранена модель
output_dir = "./saved_model_ver2"

# Создание архива ZIP
shutil.make_archive("saved_model_ver2", 'zip', output_dir)

# Скачивание файла
from google.colab import files
files.download("saved_model_ver2.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained("./saved_model_ver2")

In [ ]:
import torch

# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="./saved_model_ver2",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

In [ ]:
# Перемещение модели на GPU или CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
# Входные параметры
rubrics = "кафе"
rating = 5

In [ ]:
# Формирование входного текста
input_text = f"rubrics = {rubrics}\nrating = {rating}\n"

In [ ]:
# Токенизация текста
inputs = tokenizer(input_text, return_tensors="pt").to(device)

In [ ]:
output = model.generate(
    **inputs,
    max_length=40,
    temperature=1.9,
    do_sample=True,
    num_return_sequences=3,
    eos_token_id=tokenizer.eos_token_id,  # Установка токена конца последовательности
    pad_token_id=tokenizer.eos_token_id,  # Использовать тот же токен для паддинга
    early_stopping=True,  # Раннее завершение на токене конца
    top_k=30,         # Уменьшаем количество рассматриваемых верхних k слов
    top_p=0.8,        # Уменьшаем "ядерность" распределения
)

In [ ]:
# Декодирование всех сгенерированных текстов
for i, sequence in enumerate(output):
    generated_text = tokenizer.decode(sequence, skip_special_tokens=True)
    print(f"Generated text {i + 1}:")
    print(generated_text)
    print()

Generated text 1:
rubrics = кафе
rating = 5
  месторасположение: это очень приятный дом  дом отдыха со всеми его недостатками на фоне шикарно оформлённого здания с потрясающим видом

Generated text 2:
rubrics = кафе
rating = 5
Category: 2. Review: с таким лицом мы приехали в клуб и в самом начале 20х  не смотря на свой молодой возраст 

Generated text 3:
rubrics = кафе
rating = 5
  tip  try: 30723  3103023 заказывали роллы с креветками  2 шт  роллы

